# Dropout（CIFAR10を用いた物体認識）

---
## 目的
畳み込みニューラルネットワーク (Convolutional Neural Network; CNN) を用いてCIFAR10データセットに対する物体認識を行う．
その際，Dropoutを用いることで，認識性能がどのように変化するかを確認する．

この演習では，深層学習のフレームワークであるPyTorchを用いて，実習を行う．

## 対応するチャプター
* 7.12: ドロップアウト
* 8.1.3: バッチアルゴリズムとミニバッチアルゴリズム
* 8.3.1: 確率的勾配降下法
* 9.1: 畳み込み処理
* 9.3: プーリング


## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
実験にはこれまでに使用したNumpyに加えて，深層学習ライブラリであるPyTorchを使用します．
使用するクラス，関数は以下の通りです．

* `numpy`（説明は割愛）
* `torch`は深層学習を使用するためのPythonライブラリ
* `torchvision`はtorch上で使用する画像認識のデータセットやネットワークモデルを呼び出すためのライブラリ

In [ ]:
# モジュールのインポート
from time import time
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込み
CIFAR10データセットを読み込みます．

読み込んだ学習データのサイズを確認します．
学習データは5万枚，1つのデータサイズは3x32x32の画像のような形式となっています．
これは32x32ピクセルのカラー画像という意味になります．

In [ ]:
train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(train_data)
print(test_data)

### CIFAR10データセットの表示
CIFAR10データセットに含まれる画像を表示してみます．
ここでは，matplotlibを用いて複数の画像を表示させるプログラムを利用します．

In [ ]:
import matplotlib.pyplot as plt

cols = 10
rows = 2

plt.clf()
fig = plt.figure(figsize=(14, 4.8))
for r in range(rows):
    for c in range(cols):
        ax = fig.add_subplot(r+1, cols, c+1)
        ax.imshow(train_data[c+r*cols][0].permute(1, 2, 0))
        ax.set_axis_off()
plt.show()

## ネットワークモデルの定義
畳み込みニューラルネットワークを定義します．

ここでは，畳み込み層2層，全結合層3層から構成されるネットワークとします．

1層目の畳み込み層は入力チャンネル数が1，出力する特徴マップ数が16，畳み込むフィルタサイズが3x3です．
2層目の畳み込み層は入力チャネル数が16，出力する特徴マップ数が32，畳み込むフィルタサイズは同じく3x3です．
1つ目の全結合層は入力ユニット数は特徴マップのサイズから，`8*8*32`とし，出力は1024としています．
次の全結合層入力，出力共に1024，出力層は入力が1024，出力が10です．
また，ネットワーク内で使用する活性化関数を`self.act`に，プーリングを`self.pool`に設定します．
そして，全結合層部分で使用するDropoutをそれぞれ，`self.d_out1`, `self.d_out2`に設定します．
この時の`p`はDropout率を表し結合を削除する割合を決定します．
これらの各層の構成を`__init__`関数で定義します．

次に，`forward`関数では，定義した層を接続して処理するように記述します．
`forward`関数の引数`x`は入力データです．
それを`__init__`関数で定義した`conv1`に与え，その出力を活性化関数である`act`関数に与えます．
そして，その出力を`pool`に入力して，プーリング処理結果を`h`として出力します．
`h`は`conv2`に与えられて畳み込み処理とプーリング処理を行います．
そして，出力`h`を`l1`に与えて全結合層の処理を行います．
この全結合層部分で，Dropoutを適用します．
最終的に`l3`の全結合層の処理を行った出力`h`を戻り値としています．

In [ ]:
class CNN(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.l1 = nn.Linear(8 * 8 * 32, 1024)
        self.l2 = nn.Linear(1024, 1024)
        self.l3 = nn.Linear(1024, 10)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.d_out1 = nn.Dropout(p=p)
        self.d_out2 = nn.Dropout(p=p)
    
    def forward(self, x):
        h = self.pool(self.act(self.conv1(x)))
        h = self.pool(self.act(self.conv2(h)))
        h = h.view(h.size()[0], -1)
        h = self.d_out1(self.act(self.l1(h)))
        h = self.d_out2(self.act(self.l2(h)))
        h = self.l3(h)
        return h

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．
まず，学習時に使用するDropout率を設定し，その値を用いてネットワークモデルを作成します．


ここでは，GPUを用いた計算が可能な場合には，modelをGPUで計算するように設定する`cuda()`関数を利用しています．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．
引数として，学習するネットワークのパラメータ，学習率，モーメンタムを設定します．

In [ ]:
# ドロップアウト率pを設定
p = 0.5

model = CNN(p=p)
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を64，学習エポック数を10とします．
CIFAR10の学習データサイズを取得し，1エポック内における更新回数を求めます．

次にデータローダーを定義します．
データローダーでは，上で読み込んだデータセット（`train_data`）を用いて，for文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．
この時，`shuffle=True`と設定することで，読み込むデータを毎回ランダムに指定します．


次に，誤差関数を設定します．
今回は，分類問題をあつかうため，クロスエントロピー誤差を計算するための`CrossEntropyLoss`を`criterion`として定義します．

学習を開始します．

学習を始める前に，`model.train()`を実行することで，ネットワークの演算を学習モードへ変更します．
学習モードへ変更することで，ネットワーク内の演算のうち，学習とテストで挙動が変化する演算を一括で変更することが可能です．
今回のネットワークではDropoutが該当し，学習モードの場合はDropoutが適用され，評価モード（詳細は後ほど紹介します）では，Dropoutによる結合の削除を行わずに推論を行うことが可能です．

各更新において，学習用データと教師データをそれぞれ`image`と`label`とします．
学習モデルにimageを与えて各クラスの確率yを取得します．
各クラスの確率yと教師ラベルtとの誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，step関数ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)
        
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

## テスト
学習したネットワークモデルを用いて評価を行います．

ここで，`model.eval()`を実行することで，ネットワークを評価モードへ変更し，Dropoutの処理を行わないようにすることが可能です．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## 課題
1. ドロップアウト率やドロップアウトを適用する層を変更して，認識精度の変化を確認しましょう